## Данный пример демонстрирует создание нейросети (НС) при помощи интерпретатора специального языка (названия пока нет)

Шаги создания модели:
1. Написать скрипт с архитектурой НС.
2. Создать экземпляр парсера Parser().
3. Создать модели pytorch из скрипта при помощи функции from_str().
4. Извлечь готовый модуль из результата для использования в вычислениях.

In [ ]:
import torch
from torchview import draw_graph
from  generator.interpreter import Interpreter
import generator.bricks as bricks
import sys
import time
import torch.nn.functional as F

In [ ]:
# Тест создания моделей из выражения строки
device = "cuda"

# Примеры
examples = dict(
    s1 = "output={ {@4->relu+@8->relu}^2 }%2->@16->softmax->linear(5);",
    s2 = "output={ {@16->relu+@16->sigmoid}^4 }%8->@16;",
    s3 = "output={ {@64->relu}^16 }%16;",
    s4 = "output = linear(5) -> softmax;",
    s5 = "output = { @5->@20 + @10->@20 + @20 } -> softmax;",
    s6 = """
        y = @64 + @64;          # y - параллельно соединены x и модуль из 64 нейронов
        z = @8 -> y;            # z - x последовательно соединен с y
        w = @8 ^ 4;             # w - 4 слоя по 8 нейронов последовательно соединены
        a = {@16 + @16} % 2;    # a - параллельно соединены два модуля x
        output = z -> w -> a -> {{@8 -> relu + @8 -> relu} ^ 2} % 2 -> @16 -> softmax;
    """
)
# создаем парсер
parser = Interpreter()

# Отмечаем время старта
start_time = time.time()

# создаем модели примеров
scripts = {name: parser.parse(s) for name, s in examples.items()}

# Отмечаем время окончания создания модели
end_time = time.time()

# Результат работы парсера - набор модулей models, в которых храняться модели.
# Чтобы использовать модель - мы можем обратиться к ней по имени соответствующей переменной из скрипта.
model = scripts["s6"].get("output").to(device)

print(f"Время обработки всех скриптов: {end_time-start_time}")

In [ ]:
# Подсчитаем размер модели
from generator.visualizers import model_input_shape, model_params_count


input_shape = (1,1)

# Тестируем работу модели на тестовом тензоре
x = torch.randn(input_shape).to(device)
y = model.to(device)(x)
print(f"Результат:\n{y}")

params_count = model_params_count(model)
print(f"Параметров: {params_count}")

print(f"Размерность входного тензора: {y.shape}")
print(f"Размерность выходного тензора: {y.shape}")

In [ ]:
# Получим один из элементов модели по идентификатору подмодуля
print(f"Подсеть: \n{model.get_submodule('left')}")


In [ ]:
# Обратная конвертация модели в выражение
# Это выражение не является полноценным скриптом, т.к. не является выражением присвоения
model.expr_str(expand=True)


In [ ]:
# Нарисуем диаграмму модели
from generator.visualizers import draw_model


input_size = (1, 10)
pic_path = './pic'
graph_name = 'test'
model_graph = draw_model(model, graph_name, pic_path)

print(f"Изображение сохранено в {pic_path}/{graph_name}.png")
model_graph.resize_graph(scale=3)
model_graph.visual_graph.view(graph_name)

In [ ]:
# Пример того, как можно использовать в вычислениях отдельный подмодуль модели
# Тестовый входной тензор для модели
input_size = (1, 8)
x = torch.randn(input_size).to(device)

# Так как в синтаксисе операции '+' и '->' являются бинарными, 
# построенные из таких выражений подмодули имеют имена left и right
# Извлечем элемент left.right из модели
chunk = model.left.right
print(chunk(x))

graph_name='left.right'
model_graph = draw_model(chunk, graph_name, pic_path)

# Более того, мы можем менять структуру, например, операцией decompose()
# Разделим на две части блок chunk
left, right = chunk.decompose()

# Создадим новый модуль как соединение left и  right
new_chunk = bricks.Connector(left, right).to("cuda")
print(new_chunk(x))

graph_name='decomose'
model_graph = draw_model(chunk, graph_name, pic_path)
print()
print(f"Результат разделения и склейки сохранен в {pic_path}/{graph_name}")

In [ ]:
print(f"Было:\n{chunk}\n\n")
print(f"После chunk.decompose() и briks.Connector стало:\n{new_chunk}")